Load necessary libraries

In [39]:
# Core libraries
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Data processing and analysis
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
from typing import Tuple, List, Optional, NamedTuple
from dataclasses import dataclass, asdict




# Progress tracking and logging
from tqdm import tqdm
import logging

# Random seed
import random

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Load and preprocess data

In [40]:
# Core libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib
from typing import Tuple, List, Optional, NamedTuple
from dataclasses import dataclass
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class DataConfig:
    seq_length: int = 3
    start_season: int = 2002
    input_features: List[str] = None
    train_ratio: float = 0.7
    valid_ratio: float = 0.2
    batch_size: int = 32
    
    def __post_init__(self):
        if self.input_features is None:
            self.input_features = [
                'Age', 'ERA','FIP', 'SIERA', 'K%', 'BB%', 'HR/9', #'BABIP', 'LOB%',
                'SwStr%', 'Contact%', 'O-Swing%', 'Z-Contact%',
                'F-Strike%', 'Zone%', 'CSW%', 'CStr%',
                'GB%', 'FB%', 'IFFB%', 'HR/FB',
                'Soft%', 'Med%', 'Hard%', 'FBv', 'IP', 'GS', 'G'
                #'Stuff+', 'Location+', 'Pitching+', 
                #'EV', 'LA', 'Barrel%', 'HardHit%'
            ]

@dataclass
class PitcherConfig:
    role: str  
    min_ip: int
    min_games: int 
    gs_rate_threshold: float
    
    @classmethod
    def get_sp_config(cls):
        return cls(role='SP', min_ip=60, min_games=10, gs_rate_threshold=0.7)
    
    @classmethod
    def get_rp_config(cls):
        return cls(role='RP', min_ip=15, min_games=15, gs_rate_threshold=0.2)

class DataBatch(NamedTuple):
    train: TensorDataset
    valid: TensorDataset
    test: TensorDataset

def prepare_sequences(df: pd.DataFrame, 
                     input_features: List[str],
                     seq_length: int) -> Tuple[np.ndarray, np.ndarray]:
    sequences = []
    masks = []
    
    for player_id, player_data in df.groupby('IDfg'):
        player_data = player_data.sort_values('Season')
        
        if len(player_data) < 1:
            continue
            
        for end_idx in range(len(player_data)):
            start_idx = max(0, end_idx - seq_length + 1)
            history = player_data.iloc[start_idx:end_idx+1]
            
            if end_idx < len(player_data) - 1:
                target = player_data.iloc[end_idx + 1][input_features].values.astype(np.float32)
                
                if len(history) == seq_length:
                    sequence = history[input_features].values.astype(np.float32)
                    # Explicitly reshape mask to be [seq_length]
                    mask = np.ones(seq_length, dtype=np.int64).reshape(-1)
                else:
                    padding_needed = seq_length - len(history)
                    first_year = history.iloc[0][input_features].values.astype(np.float32)
                    padding = np.tile(first_year, (padding_needed, 1))
                    sequence = np.vstack([padding, history[input_features].values])
                    
                    # Explicitly reshape mask to be [seq_length]
                    mask = np.zeros(seq_length, dtype=np.int64).reshape(-1)
                    mask[padding_needed:] = 1
                
                # Add validation
                assert sequence.shape == (seq_length, len(input_features)), \
                    f"Sequence shape {sequence.shape} != ({seq_length}, {len(input_features)})"
                assert mask.shape == (seq_length,), \
                    f"Mask shape {mask.shape} != ({seq_length},)"
                
                sequences.append((sequence, target))
                masks.append(mask)
    
    return np.array(sequences, dtype=object), np.array(masks)




def split_data(sequences: np.ndarray, 
               masks: np.ndarray,
               train_ratio: float = 0.7,
               valid_ratio: float = 0.2) -> Tuple:
    n = len(sequences)
    indices = np.random.permutation(n)
    
    train_size = int(n * train_ratio)
    valid_size = int(n * valid_ratio)
    
    train_idx = indices[:train_size]
    valid_idx = indices[train_size:train_size + valid_size]
    test_idx = indices[train_size + valid_size:]
    
    train_sequences = sequences[train_idx]
    valid_sequences = sequences[valid_idx]
    test_sequences = sequences[test_idx]
    
    train_masks = masks[train_idx]
    valid_masks = masks[valid_idx]
    test_masks = masks[test_idx]
    
    return (train_sequences, train_masks), (valid_sequences, valid_masks), (test_sequences, test_masks)

def to_tensor(sequences: List[Tuple], masks: List[torch.Tensor]) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    X = torch.FloatTensor(np.array([s[0] for s in sequences]))  
    y = torch.FloatTensor(np.array([s[1] for s in sequences]))  
    masks = torch.LongTensor(np.array(masks))  
    
    print("\nShapes in to_tensor:")
    print(f"X: {X.shape}")
    print(f"masks: {masks.shape}")
    print(f"y: {y.shape}")
    
    return X, masks, y

def prepare_role_specific_data(
    file_path: str,
    data_config: DataConfig,
    pitcher_config: PitcherConfig
) -> Tuple[DataBatch, MinMaxScaler]:
    logger.info(f"Loading and preparing {pitcher_config.role} data from {file_path}")
    
    try:
        # Load and preprocess data
        df = pd.read_csv(file_path)
        required_cols = ['IDfg', 'Season', 'GS', 'G', 'IP'] + data_config.input_features
        missing_cols = set(required_cols) - set(df.columns)
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")
        
        # Basic cleaning
        df = df.dropna(subset=['IDfg', 'Season', 'GS', 'G', 'IP'])
        df = df[df['Season'] >= data_config.start_season]
        
        # Calculate role metrics
        df['GS_rate'] = df['GS'] / df['G']
        
        # Apply role-specific filters
        role_mask = (
            (df['GS_rate'] >= pitcher_config.gs_rate_threshold) if pitcher_config.role == 'SP'
            else (df['GS_rate'] < pitcher_config.gs_rate_threshold)
        )
        df = df[
            role_mask &
            (df['IP'] >= pitcher_config.min_ip) &
            (df['G'] >= pitcher_config.min_games)
        ]
        
        if len(df) == 0:
            raise ValueError(f"No {pitcher_config.role} data remains after filtering")
        
        # Handle missing values in features
        feature_df = df[data_config.input_features].copy()
        feature_means = feature_df.mean()
        feature_df = feature_df.fillna(feature_means)
        
        # Scale features
        scaler = MinMaxScaler(feature_range=(-1, 1))
        df[data_config.input_features] = scaler.fit_transform(feature_df)
        
        # Create sequences
        sequences, masks = prepare_sequences(
            df=df,
            input_features=data_config.input_features,
            seq_length=data_config.seq_length
        )
        
        # Split and convert to tensors
        train_data, valid_data, test_data = split_data(
            sequences=sequences,
            masks=masks,
            train_ratio=data_config.train_ratio,
            valid_ratio=data_config.valid_ratio
        )
        
        train_tensors = to_tensor(*train_data)
        valid_tensors = to_tensor(*valid_data)
        test_tensors = to_tensor(*test_data)
        
        data_batch = DataBatch(
            train=TensorDataset(*train_tensors),
            valid=TensorDataset(*valid_tensors),
            test=TensorDataset(*test_tensors)
        )
        
        logger.info(f"Successfully prepared {pitcher_config.role} data:")
        logger.info(f"Train: {len(data_batch.train)}")
        logger.info(f"Valid: {len(data_batch.valid)}")
        logger.info(f"Test: {len(data_batch.test)}")
        
        return data_batch, scaler
        
    except Exception as e:
        logger.error(f"Error processing {pitcher_config.role} data: {str(e)}")
        raise

# Initialize and run
config = DataConfig()
sp_config = PitcherConfig.get_sp_config()
rp_config = PitcherConfig.get_rp_config()

# Process data
sp_data, sp_scaler = prepare_role_specific_data(
    '../data/mlb_pitching_data_2000_2024.csv',
    config,
    sp_config
)

rp_data, rp_scaler = prepare_role_specific_data(
    '../data/mlb_pitching_data_2000_2024.csv',
    config,
    rp_config
)

# Create DataLoaders
sp_train_loader = DataLoader(sp_data.train, batch_size=config.batch_size, shuffle=True)
sp_valid_loader = DataLoader(sp_data.valid, batch_size=config.batch_size)
sp_test_loader = DataLoader(sp_data.test, batch_size=config.batch_size)

rp_train_loader = DataLoader(rp_data.train, batch_size=config.batch_size, shuffle=True)
rp_valid_loader = DataLoader(rp_data.valid, batch_size=config.batch_size)
rp_test_loader = DataLoader(rp_data.test, batch_size=config.batch_size)

# Save scalers
joblib.dump(sp_scaler, 'sp_scaler.pkl')
joblib.dump(rp_scaler, 'rp_scaler.pkl')

INFO:__main__:Loading and preparing SP data from ../data/mlb_pitching_data_2000_2024.csv
INFO:__main__:Successfully prepared SP data:
INFO:__main__:Train: 1744
INFO:__main__:Valid: 498
INFO:__main__:Test: 250
INFO:__main__:Loading and preparing RP data from ../data/mlb_pitching_data_2000_2024.csv



Shapes in to_tensor:
X: torch.Size([1744, 3, 26])
masks: torch.Size([1744, 3])
y: torch.Size([1744, 26])

Shapes in to_tensor:
X: torch.Size([498, 3, 26])
masks: torch.Size([498, 3])
y: torch.Size([498, 26])

Shapes in to_tensor:
X: torch.Size([250, 3, 26])
masks: torch.Size([250, 3])
y: torch.Size([250, 26])


INFO:__main__:Successfully prepared RP data:
INFO:__main__:Train: 2956
INFO:__main__:Valid: 844
INFO:__main__:Test: 423



Shapes in to_tensor:
X: torch.Size([2956, 3, 26])
masks: torch.Size([2956, 3])
y: torch.Size([2956, 26])

Shapes in to_tensor:
X: torch.Size([844, 3, 26])
masks: torch.Size([844, 3])
y: torch.Size([844, 26])

Shapes in to_tensor:
X: torch.Size([423, 3, 26])
masks: torch.Size([423, 3])
y: torch.Size([423, 26])


['rp_scaler.pkl']

Define Attention head mechanism

In [41]:
class MultiHeadAttention(nn.Module):
    def __init__(
        self, 
        hidden_size: int,
        num_heads: int = 8,
        dropout: float = 0.1,
        bias: bool = True
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads
        self.scaling = self.head_dim ** -0.5
        
        assert self.head_dim * num_heads == hidden_size, "hidden_size must be divisible by num_heads"
        
        # Linear projections
        self.q_proj = nn.Linear(hidden_size, hidden_size, bias=bias)
        self.k_proj = nn.Linear(hidden_size, hidden_size, bias=bias)
        self.v_proj = nn.Linear(hidden_size, hidden_size, bias=bias)
        self.out_proj = nn.Linear(hidden_size, hidden_size, bias=bias)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # Initialize parameters
        self._reset_parameters()
    
    def _reset_parameters(self):
        # Use Xavier uniform initialization
        nn.init.xavier_uniform_(self.q_proj.weight)
        nn.init.xavier_uniform_(self.k_proj.weight)
        nn.init.xavier_uniform_(self.v_proj.weight)
        nn.init.xavier_uniform_(self.out_proj.weight)
        if self.q_proj.bias is not None:
            nn.init.zeros_(self.q_proj.bias)
            nn.init.zeros_(self.k_proj.bias)
            nn.init.zeros_(self.v_proj.bias)
            nn.init.zeros_(self.out_proj.bias)
    
    def forward(
        self,
        query: torch.Tensor,
        key: Optional[torch.Tensor] = None,
        value: Optional[torch.Tensor] = None,
        key_padding_mask: Optional[torch.Tensor] = None,
        need_weights: bool = False
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        # Set key and value to query if not provided
        if key is None:
            key = query
        if value is None:
            value = query
            
        batch_size, seq_len, _ = query.size()
        
        # Project inputs
        q = self.q_proj(query)
        k = self.k_proj(key)
        v = self.v_proj(value)
        
        # Reshape for multi-head attention
        q = q.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Compute attention scores
        attn_weights = torch.matmul(q, k.transpose(-2, -1)) * self.scaling
        
        # Apply key padding mask if provided
        if key_padding_mask is not None:
            attn_weights = attn_weights.masked_fill(
                key_padding_mask.unsqueeze(1).unsqueeze(2),
                float('-inf')
            )
        
        # Apply softmax and dropout
        attn_weights = F.softmax(attn_weights, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # Get attention output
        attn_output = torch.matmul(attn_weights, v)
        
        # Reshape and project output
        attn_output = attn_output.transpose(1, 2).contiguous()
        attn_output = attn_output.view(batch_size, seq_len, self.hidden_size)
        attn_output = self.out_proj(attn_output)
        
        if need_weights:
            return attn_output, attn_weights
        return attn_output, None

Define model arch

In [42]:
class ResidualBlock(nn.Module):
    def __init__(self, hidden_size: int, dropout: float = 0.1):
        super().__init__()
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.layers = nn.Sequential(
            nn.Linear(hidden_size, hidden_size * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size * 4, hidden_size)
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x + self.layers(self.layer_norm(x))

class ImprovedLSTM(nn.Module):
    def __init__(
        self, 
        input_size: int,
        hidden_size: int = 512,
        num_layers: int = 4,
        output_size: int = None,
        dropout: float = 0.3,
        bidirectional: bool = True,
        num_heads: int = 8,
        seq_length: int = 5
    ):
        super().__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size // 2 if bidirectional else hidden_size
        self.num_layers = num_layers
        self.output_size = output_size or input_size
        self.bidirectional = bidirectional
        self.directions = 2 if bidirectional else 1
        
        # Learned embeddings for padding and position
        self.pad_token = nn.Parameter(torch.randn(1, 1, input_size))
        self.pos_encoder = nn.Parameter(torch.randn(1, seq_length, self.hidden_size))
        
        # Input projection
        self.input_projection = nn.Sequential(
            nn.Linear(input_size, self.hidden_size),
            nn.LayerNorm(self.hidden_size),
            nn.GELU(),
            nn.Dropout(dropout/2)
        )
        
        # Bidirectional LSTM layers
        self.lstm_layers = nn.ModuleList([
            nn.ModuleDict({
                'lstm': nn.LSTM(
                    self.hidden_size * self.directions if i > 0 else self.hidden_size,
                    self.hidden_size,
                    num_layers=1,
                    batch_first=True,
                    bidirectional=bidirectional
                ),
                'norm': nn.LayerNorm(self.hidden_size * self.directions, eps=1e-12),
                'dropout': nn.Dropout(dropout/2)
            }) for i in range(self.num_layers)
        ])
        
        # Multi-head attention
        self.attention = MultiHeadAttention(
            self.hidden_size * self.directions,
            num_heads=num_heads,
            dropout=dropout/2
        )
        
        # Context layer
        self.context_layer = nn.Sequential(
            nn.Linear(self.hidden_size * self.directions, hidden_size),
            nn.LayerNorm(hidden_size, eps=1e-12),
            nn.GELU(),
            nn.Dropout(dropout/2)
        )
        
        # Output projection
        self.output_projection = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.LayerNorm(hidden_size, eps=1e-12),
            nn.GELU(),
            nn.Dropout(dropout/2),
            nn.Linear(hidden_size, self.output_size)
        )

    def forward(self, x: torch.Tensor, lengths: torch.Tensor) -> torch.Tensor:
        batch_size, seq_len, _ = x.size()
        
        # Ensure lengths are long tensors immediately
        lengths = lengths.long()
        
        # Replace padding with learned token 
        padding_mask = (x.sum(dim=-1) == 0).unsqueeze(-1)
        x = torch.where(padding_mask, self.pad_token.expand(batch_size, seq_len, -1), x)
        
        # Create attention mask for valid positions - ensure boolean type
        attention_mask = torch.arange(seq_len, device=x.device)[None, :] < lengths[:, None]
        attention_mask = attention_mask.bool()  # Explicit cast to boolean
        
        # Project input and add positional encoding
        x = self.input_projection(x)
        x = x + self.pos_encoder[:, :seq_len, :]
        
        # Store layer outputs
        layer_outputs = []
        
        # Process LSTM layers
        for layer in self.lstm_layers:
            packed_x = pack_padded_sequence(
                x, 
                lengths.cpu(),  # lengths are already long type
                batch_first=True,
                enforce_sorted=False
            )
            
            lstm_out, _ = layer['lstm'](packed_x)
            lstm_out, _ = pad_packed_sequence(
                lstm_out,
                batch_first=True,
                total_length=seq_len
            )
            
            lstm_out = layer['norm'](lstm_out)
            lstm_out = layer['dropout'](lstm_out)
            
            if lstm_out.size(-1) == x.size(-1):
                x = x + lstm_out
            else:
                x = lstm_out
                
            layer_outputs.append(x)
        
        # Apply attention with masking
        attended, _ = self.attention(
            x, x, x,
            key_padding_mask=~attention_mask
        )
        
        # Get sequence context
        context = self.context_layer(attended.mean(dim=1))
        
        # Get final states
        batch_indices = torch.arange(batch_size, device=x.device, dtype=torch.long)
        last_states = x[batch_indices, lengths - 1]  # lengths already long type
        
        # Combine context and last states
        combined = torch.cat([context, last_states], dim=-1)
        
        # Project to output size
        output = self.output_projection(combined)
        
        return output

Instantiate model

In [43]:
# Model Configuration
@dataclass
class Config:
    """Advanced configuration for LSTM-based baseball statistics prediction."""
    
    # Dynamic sizes from data
    input_size: int = None
    output_size: int = None
    
    # Model Architecture 
    hidden_size: int = 512
    num_layers: int = 6
    num_heads: int = 4
    bidirectional: bool = True
    attention_dropout: float = 0.1
    residual_dropout: float = 0.2
    layer_norm_eps: float = 1e-5
    seq_length: int = 3
    
    # Training Parameters
    batch_size: int = 32
    dropout: float = 0.3
    learning_rate: float = 1e-3
    weight_decay: float = 1e-5
    gradient_clip: float = 1.0
    num_epochs: int = 50
    warmup_epochs: int = 5
    
    # Learning Rate Schedule
    lr_schedule: str = 'cosine'
    min_lr: float = 1e-6
    lr_decay_rate: float = 0.1
    lr_patience: int = 5
    
    # Early Stopping
    early_stopping_patience: int = 10
    early_stopping_min_delta: float = 1e-4
    
    # Loss Function Parameters
    diversity_alpha: float = 0.1
    consistency_beta: float = 0.05
    
    # Hardware Optimization
    mixed_precision: bool = True
    num_workers: int = 0
    pin_memory: bool = True
    
    # Logging
    log_interval: int = 100
    checkpoint_interval: int = 1
    
    def __init__(self, X_train: torch.Tensor, y_train: torch.Tensor):
        self.input_size = X_train.shape[2]
        self.output_size = y_train.shape[1]
        self._validate_config()
        self._log_config()
    
    def _validate_config(self) -> None:
        assert self.hidden_size % self.num_heads == 0, \
            "Hidden size must be divisible by number of attention heads"
        assert self.hidden_size >= self.input_size, \
            "Hidden size must be greater than or equal to input size"
        assert 0 <= self.dropout <= 1, "Dropout must be between 0 and 1"
        assert self.num_layers >= 1, "Must have at least one LSTM layer"
    
    def _log_config(self) -> None:
        logger.info("Model Configuration:")
        for key, value in asdict(self).items():
            logger.info(f"{key}: {value}")
    
    @property
    def device(self) -> torch.device:
        return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [44]:
# Initialize SP model and training
try:
    # Initialize SP config
    sp_config = Config(next(iter(sp_train_loader))[0], next(iter(sp_train_loader))[2])
    
    logger.info("Training Starting Pitcher Model")
    sp_model = ImprovedLSTM(
        input_size=sp_config.input_size,
        hidden_size=sp_config.hidden_size,
        num_layers=sp_config.num_layers,
        output_size=sp_config.output_size,
        dropout=sp_config.dropout,
        bidirectional=sp_config.bidirectional,
        num_heads=sp_config.num_heads,
        seq_length=sp_config.seq_length
    ).to(sp_config.device)
    
    sp_optimizer = optim.AdamW(
        sp_model.parameters(),
        lr=sp_config.learning_rate,
        weight_decay=sp_config.weight_decay
    )
    
    sp_scheduler = optim.lr_scheduler.OneCycleLR(
        sp_optimizer,
        max_lr=sp_config.learning_rate,
        epochs=sp_config.num_epochs,
        steps_per_epoch=len(sp_train_loader),
        pct_start=sp_config.warmup_epochs / sp_config.num_epochs,
        anneal_strategy='cos',
        final_div_factor=1e3
    )
    
    sp_criterion = nn.MSELoss()
    
    # Initialize RP model and training
    rp_config = Config(next(iter(rp_train_loader))[0], next(iter(rp_train_loader))[2])
    
    logger.info("Training Relief Pitcher Model")
    rp_model = ImprovedLSTM(
        input_size=rp_config.input_size,
        hidden_size=rp_config.hidden_size,
        num_layers=rp_config.num_layers,
        output_size=rp_config.output_size,
        dropout=rp_config.dropout,
        bidirectional=rp_config.bidirectional,
        num_heads=rp_config.num_heads,
        seq_length=rp_config.seq_length
    ).to(rp_config.device)
    
    rp_optimizer = optim.AdamW(
        rp_model.parameters(),
        lr=rp_config.learning_rate,
        weight_decay=rp_config.weight_decay
    )
    
    rp_scheduler = optim.lr_scheduler.OneCycleLR(
        rp_optimizer,
        max_lr=rp_config.learning_rate,
        epochs=rp_config.num_epochs,
        steps_per_epoch=len(rp_train_loader),
        pct_start=rp_config.warmup_epochs / rp_config.num_epochs,
        anneal_strategy='cos',
        final_div_factor=1e3
    )
    
    rp_criterion = nn.MSELoss()

except Exception as e:
    logger.error(f"Error during model initialization: {str(e)}")
    raise

INFO:__main__:Model Configuration:
INFO:__main__:input_size: 26
INFO:__main__:output_size: 26
INFO:__main__:hidden_size: 512
INFO:__main__:num_layers: 6
INFO:__main__:num_heads: 4
INFO:__main__:bidirectional: True
INFO:__main__:attention_dropout: 0.1
INFO:__main__:residual_dropout: 0.2
INFO:__main__:layer_norm_eps: 1e-05
INFO:__main__:seq_length: 3
INFO:__main__:batch_size: 32
INFO:__main__:dropout: 0.3
INFO:__main__:learning_rate: 0.001
INFO:__main__:weight_decay: 1e-05
INFO:__main__:gradient_clip: 1.0
INFO:__main__:num_epochs: 50
INFO:__main__:warmup_epochs: 5
INFO:__main__:lr_schedule: cosine
INFO:__main__:min_lr: 1e-06
INFO:__main__:lr_decay_rate: 0.1
INFO:__main__:lr_patience: 5
INFO:__main__:early_stopping_patience: 10
INFO:__main__:early_stopping_min_delta: 0.0001
INFO:__main__:diversity_alpha: 0.1
INFO:__main__:consistency_beta: 0.05
INFO:__main__:mixed_precision: True
INFO:__main__:num_workers: 0
INFO:__main__:pin_memory: True
INFO:__main__:log_interval: 100
INFO:__main__:chec

INFO:__main__:Model Configuration:
INFO:__main__:input_size: 26
INFO:__main__:output_size: 26
INFO:__main__:hidden_size: 512
INFO:__main__:num_layers: 6
INFO:__main__:num_heads: 4
INFO:__main__:bidirectional: True
INFO:__main__:attention_dropout: 0.1
INFO:__main__:residual_dropout: 0.2
INFO:__main__:layer_norm_eps: 1e-05
INFO:__main__:seq_length: 3
INFO:__main__:batch_size: 32
INFO:__main__:dropout: 0.3
INFO:__main__:learning_rate: 0.001
INFO:__main__:weight_decay: 1e-05
INFO:__main__:gradient_clip: 1.0
INFO:__main__:num_epochs: 50
INFO:__main__:warmup_epochs: 5
INFO:__main__:lr_schedule: cosine
INFO:__main__:min_lr: 1e-06
INFO:__main__:lr_decay_rate: 0.1
INFO:__main__:lr_patience: 5
INFO:__main__:early_stopping_patience: 10
INFO:__main__:early_stopping_min_delta: 0.0001
INFO:__main__:diversity_alpha: 0.1
INFO:__main__:consistency_beta: 0.05
INFO:__main__:mixed_precision: True
INFO:__main__:num_workers: 0
INFO:__main__:pin_memory: True
INFO:__main__:log_interval: 100
INFO:__main__:chec

Define training loop

In [45]:
def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    valid_loader: DataLoader,
    config: Config,
    optimizer: optim.Optimizer,
    scheduler: optim.lr_scheduler._LRScheduler,
    criterion: nn.Module,
    checkpoint_dir: str = './checkpoints'
) -> dict:
    logger.info(f"Starting training on device: {config.device}")
    model = model.to(config.device)
    
    # Mixed precision training
    scaler = torch.cuda.amp.GradScaler(enabled=config.mixed_precision)
    
    # Training state tracking
    best_val_loss = float('inf')
    early_stopping_counter = 0
    train_metrics = {
        'train_losses': [],
        'val_losses': [],
        'learning_rates': [],
        'best_epoch': 0
    }
    
    # Create checkpoint directory
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    for epoch in range(config.num_epochs):
        # Training phase
        model.train()
        epoch_loss = 0.0
        
        
        with tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.num_epochs}') as pbar:
            for batch_idx, (data, masks, targets) in enumerate(pbar):
                try:
                    # Move data to device and ensure correct dtypes
                    data = data.to(config.device, dtype=torch.float32)
                    masks = masks.to(config.device, dtype=torch.long)  # Ensure long type for masks
                    targets = targets.to(config.device, dtype=torch.float32)
                    
                    # Calculate sequence lengths from masks (keep as long)
                    lengths = masks.sum(1).long().clamp(min=1)  # Explicit long conversion
                    
                    # Forward pass with mixed precision
                    with torch.cuda.amp.autocast(enabled=config.mixed_precision):
                        outputs = model(data, lengths)  # lengths is now guaranteed long
                        loss = criterion(outputs, targets)
                    
                    # Backward pass with gradient scaling
                    optimizer.zero_grad(set_to_none=True)
                    scaler.scale(loss).backward()
                    
                    # Gradient clipping
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), config.gradient_clip)
                    
                    # Optimizer step with scaler
                    scaler.step(optimizer)
                    scaler.update()
                    
                    # Update scheduler
                    if scheduler is not None:
                        scheduler.step()
                    
                    # Update metrics
                    epoch_loss += loss.item()
                    current_lr = scheduler.get_last_lr()[0] if scheduler else optimizer.param_groups[0]['lr']
                    
                    # Update progress bar
                    pbar.set_postfix({
                        'loss': f'{loss.item():.3f}',
                        'lr': f'{current_lr:.2e}'
                    })
                    
                except RuntimeError as e:
                    logger.error(f"Error in batch {batch_idx}: {str(e)}")
                    raise

        # Validation phase
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for data, masks, targets in valid_loader:
                try:
                    data = data.to(config.device)
                    masks = masks.to(config.device)
                    targets = targets.to(config.device)
                    lengths = masks.sum(1).clamp(min=1).long()
                    
                    with torch.cuda.amp.autocast(enabled=config.mixed_precision):
                        outputs = model(data, lengths)
                        loss = criterion(outputs, targets)
                        val_loss += loss.item()
                        
                except RuntimeError as e:
                    logger.error(f"Error in validation: {str(e)}")
                    raise
        
        # Calculate epoch metrics
        epoch_loss /= len(train_loader)
        val_loss /= len(valid_loader)
        
        # Update training metrics
        train_metrics['train_losses'].append(epoch_loss)
        train_metrics['val_losses'].append(val_loss)
        train_metrics['learning_rates'].append(current_lr)
        
        # Model checkpointing
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            train_metrics['best_epoch'] = epoch
            early_stopping_counter = 0
            
            # Save checkpoint
            checkpoint_path = os.path.join(checkpoint_dir, 'pitcher_model.pth')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'val_loss': val_loss,
                'config': asdict(config),
                'metrics': train_metrics,
                'scaler_state_dict': scaler.state_dict()
            }, checkpoint_path)
            
            logger.info(f'New best model saved with validation loss: {val_loss:.4f}')
        else:
            early_stopping_counter += 1
        
        # Log epoch metrics
        logger.info(
            f'Epoch {epoch+1}: '
            f'Train Loss = {epoch_loss:.4f}, '
            f'Val Loss = {val_loss:.4f}, '
            f'LR = {current_lr:.2e}'
        )
        
        # Early stopping check
        if early_stopping_counter >= config.early_stopping_patience:
            logger.info(f'Early stopping triggered after {epoch+1} epochs')
            break
    
    return train_metrics



Train SP and RP models

In [46]:
import datetime
import json

# Train SP Model
try:
    logger.info("Starting SP model training...")
    sp_metrics = train_model(
        model=sp_model,
        train_loader=sp_train_loader,
        valid_loader=sp_valid_loader,
        config=sp_config,
        optimizer=sp_optimizer,
        scheduler=sp_scheduler,
        criterion=sp_criterion,
        checkpoint_dir='checkpoints/sp'
    )
    
    logger.info(f"SP Model Best Val Loss: {min(sp_metrics['val_losses']):.4f}")
    
except Exception as e:
    logger.error(f"Error training SP model: {str(e)}")
    raise

# Train RP Model
try:
    logger.info("Starting RP model training...")
    rp_metrics = train_model(
        model=rp_model,
        train_loader=rp_train_loader,
        valid_loader=rp_valid_loader,
        config=rp_config,
        optimizer=rp_optimizer,
        scheduler=rp_scheduler,
        criterion=rp_criterion,
        checkpoint_dir='checkpoints/rp'
    )
    
    logger.info(f"RP Model Best Val Loss: {min(rp_metrics['val_losses']):.4f}")
    
except Exception as e:
    logger.error(f"Error training RP model: {str(e)}")
    raise

# Save final metrics
training_results = {
    'sp_metrics': sp_metrics,
    'rp_metrics': rp_metrics,
    'timestamp': datetime.datetime.now().isoformat()
}

with open('training_results.json', 'w') as f:
    json.dump(training_results, f, indent=2)

logger.info("Training complete! Results saved to training_results.json")

INFO:__main__:Starting SP model training...
INFO:__main__:Starting training on device: cuda
C:\Users\User\AppData\Local\Temp\ipykernel_26492\2538770.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=config.mixed_precision)
Epoch 1/50:   0%|          | 0/55 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_26492\2538770.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=config.mixed_precision):
Epoch 1/50: 100%|██████████| 55/55 [00:04<00:00, 12.58it/s, loss=0.104, lr=1.32e-04]
C:\Users\User\AppData\Local\Temp\ipykernel_26492\2538770.py:94: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=config.mixed_precision):
INFO:__main

Use model to predict future years

In [47]:
def generate_pitcher_names(raw_df: pd.DataFrame, output_path: str = '../data/pitcher_names.csv'):
    """Generate and save a dataset of pitcher names and IDs"""
    try:
        # Get unique pitcher entries
        pitcher_names = raw_df[['Name', 'IDfg']].drop_duplicates()
        
        # Sort by Name for easier reference
        pitcher_names = pitcher_names.sort_values('Name')
        
        # Save to CSV
        pitcher_names.to_csv(output_path, index=False)
        logger.info(f"Saved {len(pitcher_names)} pitcher names to {output_path}")
        
        return pitcher_names
        
    except Exception as e:
        logger.error(f"Error generating pitcher names: {str(e)}")
        raise

In [54]:
def load_model_from_checkpoint(checkpoint_path: str, data_config, device: torch.device) -> nn.Module:
    """Load model with proper error handling and validation"""
    try:
        logger.info(f"Loading model from {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        
        model = ImprovedLSTM(
            input_size=len(data_config.input_features),
            hidden_size=512,
            num_layers=6,
            output_size=len(data_config.input_features),
            dropout=0.2,
            bidirectional=True,
            num_heads=4,
            seq_length=3
        ).to(device)
        
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        
        return model
        
    except Exception as e:
        logger.error(f"Error loading model: {str(e)}")
        raise

def predict_future_years(player_id, input_features, model, scaler, raw_df, player_names, pitcher_type, seq_length=3, future_years=5):
    """Modified to maintain consistent masking behavior"""
    
    # Get initial player data
    player_data = raw_df[raw_df['IDfg'] == player_id].sort_values('Season')
    if len(player_data) < 1:
        return None
        
    # Get player info and handle missing values
    player_name = player_names[player_names['IDfg'] == player_id]['Name'].iloc[0]
    last_season = player_data['Season'].max()
    last_age = player_data[player_data['Season'] == last_season]['Age'].iloc[0]
    
    # Fill missing values with player-specific means
    feature_data = player_data[input_features].fillna(player_data[input_features].mean())
    
    device = next(model.parameters()).device
    predictions_list = []
    
    # Initialize sequence with available history
    current_sequence = feature_data.values
    if len(current_sequence) < seq_length:
        first_year = current_sequence[0]
        padding = np.tile(first_year, (seq_length - len(current_sequence), 1))
        current_sequence = np.vstack([padding, current_sequence])
        # Create mask matching our training data format
        mask = np.zeros(seq_length, dtype=np.int64)
        mask[seq_length - len(feature_data):] = 1
    else:
        current_sequence = current_sequence[-seq_length:]
        mask = np.ones(seq_length, dtype=np.int64)
    
    # Generate predictions
    for year in range(1, future_years + 1):
        sequence_scaled = scaler.transform(current_sequence)
        sequence_tensor = torch.FloatTensor(sequence_scaled).unsqueeze(0).to(device)
        mask_tensor = torch.LongTensor(mask).unsqueeze(0).to(device)
        
        with torch.no_grad():
            prediction = model(sequence_tensor, mask_tensor.sum(1))
            prediction = prediction.cpu().numpy()
        
        prediction_unscaled = scaler.inverse_transform(prediction)[0]
        
        pred_dict = {
            'Name': player_name,
            'Season': last_season + year,
            'Age': last_age + year,
            'Role': pitcher_type,
            'IDfg': player_id
        }
        
        # Add predicted stats (except Age)
        for i, feature in enumerate(input_features):
            if feature != 'Age':
                pred_dict[feature] = prediction_unscaled[i]
        
        predictions_list.append(pred_dict)
        
        # Update sequence for next prediction
        next_sequence = prediction_unscaled.copy()
        age_index = input_features.index('Age')
        next_sequence[age_index] = last_age + year + 1
        current_sequence = np.vstack([current_sequence[1:], next_sequence])
        mask = np.ones(seq_length, dtype=np.int64)  # All valid for subsequent predictions
    
    return predictions_list

def predict_all_2024_pitchers(raw_df, player_names, sp_model, rp_model, sp_scaler, rp_scaler, input_features, seq_length, future_years=5):
    """Predict future years for qualified 2024 pitchers and 2023 pitchers who were injured/below threshold in 2024"""
    logger.info("Starting predictions for current and potentially injured/recovering pitchers")
    
    # Get 2024 and 2023 pitchers
    pitchers_2024 = raw_df[raw_df['Season'] == 2024].copy()
    pitchers_2023 = raw_df[raw_df['Season'] == 2023].copy()
    
    # Calculate GS rates
    pitchers_2024['GS_rate'] = pitchers_2024['GS'] / pitchers_2024['G']
    pitchers_2023['GS_rate'] = pitchers_2023['GS'] / pitchers_2023['G']
    
    # 2024 qualified SPs and RPs
    sp_ids_2024 = set(pitchers_2024[
        (pitchers_2024['IP'] >= 30) & 
        (pitchers_2024['G'] >= 6) & 
        (pitchers_2024['GS_rate'] >= 0.7)
    ]['IDfg'])
    
    rp_ids_2024 = set(pitchers_2024[
        (pitchers_2024['IP'] >= 15) & 
        (pitchers_2024['G'] >= 15) & 
        (pitchers_2024['GS_rate'] < 0.2)
    ]['IDfg'])
    
    # 2023 qualified pitchers who either:
    # 1. Don't appear in 2024
    # 2. Appear in 2024 but don't meet thresholds
    sp_ids_2023 = set(pitchers_2023[
        (pitchers_2023['IP'] >= 30) & 
        (pitchers_2023['G'] >= 6) & 
        (pitchers_2023['GS_rate'] >= 0.7) &
        (~pitchers_2023['IDfg'].isin(sp_ids_2024))  # Not qualified in 2024
    ]['IDfg'])
    
    rp_ids_2023 = set(pitchers_2023[
        (pitchers_2023['IP'] >= 15) & 
        (pitchers_2023['G'] >= 15) & 
        (pitchers_2023['GS_rate'] < 0.2) &
        (~pitchers_2023['IDfg'].isin(rp_ids_2024))  # Not qualified in 2024
    ]['IDfg'])
    
    # Combine IDs
    sp_ids = sp_ids_2024.union(sp_ids_2023)
    rp_ids = rp_ids_2024.union(rp_ids_2023)
    
    logger.info(f"Found {len(sp_ids_2024)} qualified 2024 SPs and {len(sp_ids_2023)} returning/recovering SPs")
    logger.info(f"Found {len(rp_ids_2024)} qualified 2024 RPs and {len(rp_ids_2023)} returning/recovering RPs")
    
    
    
    
    all_predictions = []
    
    # Predict SPs
    logger.info("Generating SP predictions...")
    for player_id in tqdm(sp_ids, desc="Starting Pitchers"):
        predictions = predict_future_years(
            player_id=player_id,
            input_features=input_features,
            model=sp_model,
            scaler=sp_scaler,
            raw_df=raw_df,
            player_names=player_names,
            pitcher_type='SP',
            seq_length=seq_length,
            future_years=future_years
        )
        if predictions:
            all_predictions.extend(predictions)
            
    # Predict RPs
    logger.info("Generating RP predictions...")
    for player_id in tqdm(rp_ids, desc="Relief Pitchers"):
        predictions = predict_future_years(
            player_id=player_id,
            input_features=input_features,
            model=rp_model,
            scaler=rp_scaler,
            raw_df=raw_df,
            player_names=player_names,
            pitcher_type='RP',
            future_years=future_years
        )
        if predictions:
            all_predictions.extend(predictions)
    
    if all_predictions:
        predictions_df = pd.DataFrame(all_predictions)
        
        # Save predictions by year and role
        for year in range(2025, 2025 + future_years):
            year_predictions = predictions_df[predictions_df['Season'] == year]
            
            # Split and save SP predictions
            sp_predictions = year_predictions[year_predictions['Role'] == 'SP'].sort_values('FIP')
            sp_predictions.to_csv(f'../data/generated/SP_Predictions_{year}.csv', index=False)
            
            # Split and save RP predictions
            rp_predictions = year_predictions[year_predictions['Role'] == 'RP'].sort_values('FIP')
            rp_predictions.to_csv(f'../data/generated/RP_Predictions_{year}.csv', index=False)
            
            # Display top performers
            print(f"\nTop 10 Starting Pitchers for {year}:")
            print(sp_predictions[['Name', 'Age', 'FIP', 'K%', 'BB%', 'HR/9']].head(10))
            
            print(f"\nTop 10 Relief Pitchers for {year}:")
            print(rp_predictions[['Name', 'Age', 'FIP', 'K%', 'BB%', 'HR/9']].head(10))
        
        return predictions_df
    else:
        logger.warning("No predictions were generated")
        return None


data_config = DataConfig(
    seq_length=3,
    input_features = [
                'Age', 'ERA','FIP', 'SIERA', 'K%', 'BB%', 'HR/9', #'BABIP', 'LOB%',
                'SwStr%', 'Contact%', 'O-Swing%', 'Z-Contact%',
                'F-Strike%', 'Zone%', 'CSW%', 'CStr%',
                'GB%', 'FB%', 'IFFB%', 'HR/FB',
                'Soft%', 'Med%', 'Hard%', 'FBv', 'IP', 'GS', 'G'
                #'Stuff+', 'Location+', 'Pitching+', 
                #'EV', 'LA', 'Barrel%', 'HardHit%'
            ]
)
# Execute predictions
raw_df = pd.read_csv('../data/mlb_pitching_data_2000_2024.csv')

if not os.path.exists('../data/pitcher_names.csv'):
    player_names = pd.DataFrame(raw_df[['Name', 'IDfg']].drop_duplicates()).sort_values('Name')
    player_names.to_csv('../data/pitcher_names.csv', index=False)
else:
    player_names = pd.read_csv('../data/pitcher_names.csv')

# Load models and scalers
sp_model = load_model_from_checkpoint(
    'checkpoints/sp/pitcher_model.pth',
    data_config,
    device
)

rp_model = load_model_from_checkpoint(
    'checkpoints/rp/pitcher_model.pth',
    data_config,
    device
)

sp_scaler = joblib.load('sp_scaler.pkl')
rp_scaler = joblib.load('rp_scaler.pkl')

predictions_df = predict_all_2024_pitchers(
    raw_df=raw_df,
    player_names=player_names,
    sp_model=sp_model,
    rp_model=rp_model,
    sp_scaler=sp_scaler,
    rp_scaler=rp_scaler,
    input_features=data_config.input_features,
    seq_length=data_config.seq_length,
    future_years=15
)

INFO:__main__:Loading model from checkpoints/sp/pitcher_model.pth
C:\Users\User\AppData\Local\Temp\ipykernel_26492\1688909069.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur


Top 10 Starting Pitchers for 2025:
               Name  Age       FIP        K%       BB%      HR/9
2820    Paul Skenes   23  2.910408  0.281627  0.058453  0.913400
1830   Tarik Skubal   28  3.101918  0.284603  0.055323  1.037978
540    Zack Wheeler   35  3.171725  0.292251  0.056738  1.146671
345   Tyler Glasnow   31  3.278028  0.308583  0.065443  1.109113
1650  Logan Gilbert   28  3.380732  0.262463  0.052614  1.159394
2176  Shohei Ohtani   30  3.421740  0.318581  0.077182  1.142745
2580     Chris Sale   36  3.423531  0.271796  0.057057  1.127422
1425    Pablo Lopez   29  3.440081  0.256922  0.054274  1.144219
3075  Justin Steele   29  3.466364  0.238452  0.057425  1.009789
1605    Blake Snell   32  3.480317  0.322178  0.095088  1.071287

Top 10 Relief Pitchers for 2025:
                Name  Age       FIP        K%       BB%      HR/9
7455     Jhoan Duran   27  2.818610  0.311160  0.094087  0.578536
5265      Cade Smith   26  2.842750  0.305483  0.084149  0.679507
4995    Tanner Sc

Calculate WAR and add to predictions

In [55]:
def calculate_war(df: pd.DataFrame) -> pd.DataFrame:
    """Calculate WAR using correct replacement level baseline"""
    
    def estimate_playing_time(row):
        if row['Role'] == 'SP':
            base_ip = 180
            scaled_ip = base_ip * (4.20/row['FIP'])
            return pd.Series({
                'IP': min(220, max(150, scaled_ip)),
                'G': 32,
                'GS': 30
            })
        else:
            base_ip = 65
            scaled_ip = base_ip * (4.20/row['FIP'])
            return pd.Series({
                'IP': min(80, max(50, scaled_ip)),
                'G': 65,
                'GS': 0
            })
    
    df[['IP', 'G', 'GS']] = df.apply(estimate_playing_time, axis=1)
    
    # Calculate WAR components
    league_fip = 4.20
    replacement_level_fip = 4.95  # Approximately 0.75 runs worse than league average
    
    # Runs above replacement level
    df['RAR'] = (replacement_level_fip - df['FIP']) * (df['IP'] / 9)
    
    # Calculate WAR
    df['WAR'] = df['RAR'] / 9.0
    
    # Cleanup and round
    df = df.drop(columns=['RAR'])
    df['WAR'] = df['WAR'].round(1)
    
    return df

# Process predictions
for year in range(2025, 2040):
    for role in ['SP', 'RP']:
        try:
            df = pd.read_csv(f"../data/generated/{role}_Predictions_{year}.csv")
            df_with_war = calculate_war(df)
            output_file = f"../data/generated/{role}_Predictions_{year}.csv"
            df_with_war.to_csv(output_file, index=False)
            print(f"\n{role} {year} Summary:")
            print(f"Average WAR: {df_with_war['WAR'].mean():.2f}")
            print(f"Max WAR: {df_with_war['WAR'].max():.2f}")
            
        except Exception as e:
            print(f"Error processing {year} {role}: {str(e)}")


SP 2025 Summary:
Average WAR: 1.57
Max WAR: 5.50

RP 2025 Summary:
Average WAR: 0.84
Max WAR: 2.10

SP 2026 Summary:
Average WAR: 1.40
Max WAR: 5.20

RP 2026 Summary:
Average WAR: 0.69
Max WAR: 2.00

SP 2027 Summary:
Average WAR: 1.31
Max WAR: 5.20

RP 2027 Summary:
Average WAR: 0.55
Max WAR: 1.80

SP 2028 Summary:
Average WAR: 1.20
Max WAR: 5.10

RP 2028 Summary:
Average WAR: 0.42
Max WAR: 1.60

SP 2029 Summary:
Average WAR: 1.10
Max WAR: 5.10

RP 2029 Summary:
Average WAR: 0.33
Max WAR: 1.30

SP 2030 Summary:
Average WAR: 1.02
Max WAR: 5.20

RP 2030 Summary:
Average WAR: 0.26
Max WAR: 1.00

SP 2031 Summary:
Average WAR: 0.95
Max WAR: 5.30

RP 2031 Summary:
Average WAR: 0.22
Max WAR: 0.80

SP 2032 Summary:
Average WAR: 0.89
Max WAR: 5.50

RP 2032 Summary:
Average WAR: 0.19
Max WAR: 0.70

SP 2033 Summary:
Average WAR: 0.84
Max WAR: 5.60

RP 2033 Summary:
Average WAR: 0.17
Max WAR: 0.60

SP 2034 Summary:
Average WAR: 0.79
Max WAR: 5.80

RP 2034 Summary:
Average WAR: 0.16
Max WAR: 0.50
